# Example processing script for SEVIRI data
### A more complex example than for SLSTR, using a land-mask and ERA5 wind profiles.

This script requires MSG/SEVIRI data in native format (although it can be easily edited to use HRIT data instead).

For this example, a full disk SEVIRI dataset from 26th March 2016 at 15:00 UTC was used. The data file is available via zenodo:
https://doi.org/10.5281/zenodo.7885995

The data file is copyright EUMETSAT (2016).


ERA5 data is included in the Examples directory. These files were generated using Copernicus Climate Change Service information (2022).

Output from this example notebook is available on zenodo at the same link as above.

This example assumes that input data is located in the same directory as the notebook itself, and that output data should be saved to an `out` subfolder. These may not be convenient, so you can change the directories with the `sev_fname` and `out_dir` variables below.

In this example we compute the sea surface reflectance and also the BRDF values:

    - rho_0d: Solar beam to satellite view reflectances
    - rho_0d: Solar beam to diffuse reflectances
    - rho_dv: Diffuse to satellite view reflectances
    - rho_dd: Diffuse to diffuse reflectances
    
We also apply a land/sea mask (derived from the NASA SRTM data and remapped onto the SEVIRI grid) to mask out all non-water pixels.


NOTE: This requires the netcdf and rioxarray python libraries, you must install there for this example to work!
They can be installed with:

`conda install -c conda-forge netCDF4 rioxarray` or `pip install netCDF4 rioxarray`

In [1]:
%load_ext autoreload
%autoreload 2

# If you have a powerful machine, the dask library can actually slow things down.
# These lines limit the resources dask uses, you may need to experiment with the
# `num_workers` argument to find what works best on your own machine.
import dask
dask.config.set({'array.chunk-size': '32M', 'num_workers': 4})

from dask.diagnostics import ResourceProfiler, Profiler, CacheProfiler, visualize

In [2]:
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from datetime import datetime
from netCDF4 import Dataset
import dask.array as da
import xarray as xr
import numpy as np
import pycoxmunk

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Filename of the input granule
sev_fname = './MSG3-SEVI-MSG15-0100-NA-20160326151242.278000000Z-NA.nat'

# Landmask filename
lsm_fname = './SEVIRI_LANDMASK.tiff'

# Output directory
out_dir = './out/'

# Filenames of the U and V 10m wind components, here taken from ERA5
u10_fname = './ecmwf-era5_oper_an_sfc_201603261500.10u.nc'
v10_fname = './ecmwf-era5_oper_an_sfc_201603261500.10v.nc'

# Band name to load / process
bnames = ['VIS006', 'VIS008', 'IR_016']

# Band names to save
bnames_out = ['VIS006', 'VIS008', 'IR_016',
              'cox_munk_refl_VIS006', 'cox_munk_refl_VIS008', 'cox_munk_refl_IR_016',
              'cox_munk_rhodv_VIS006', 'cox_munk_rho0v_VIS006']
              

# Cache dir for use in satpy's resample routines
# This is optional, but can help speed things up - especially for 
# geostationary satellites where the area covered is a constant region.
cache_dir = './cache/'


# A list of datasets that we want to save
out_ds_list = []
for bname in bnames:
    out_ds_list.append(bname)
    out_ds_list.append(f'cos_munk_refl_{bname}')
    out_ds_list.append(f'cos_munk_rho0d_{bname}')
    out_ds_list.append(f'cos_munk_rhodd_{bname}')

In [4]:
# A helper function needed for ECMWF processing
def load_wind(fname, var, ecm_scn=None, ftype='nc', dater=datetime(2000, 1, 1, 0, 0, 0)):
    """Load wind datasets from a file.
    
    This assumes that the winds are in individual netCDF files. It also makes a lot of
    assumptions about the data type and area covered. Thus, it will probably not work
    for anything other than the ERA5 data used for internal testing but it can be the
    basis for your own wind loading functions.
    
    Inputs:
     - fname: String, the input filename.
     - var: String, name of variable to read
     - ecm_scn: Scene, an existing scene to save data into. If None, a new Scene is created.
     - ftype: String, file type. Currently only netCDF ('nc') is supported.
     - dater: DateTime, time to set as Scene start_time.
    Returns:
      - ecm_scn: Scene, containing 'u10' and 'v10' winds.
    """
    if ftype != 'nc':
        raise ValueError("Only netCDF winds are supported at present.")
    
    # Open the netCDF file
    fid = Dataset(fname, 'r')
    # Load the variable and retrieve the lats + lons
    inv = np.array(fid[var]).squeeze()
    lat = np.array(fid['latitude']).squeeze()
    lon = np.array(fid['longitude']).squeeze()
    # ERA5 has lats in range 0 -> 360. Here we switch to -180 -> 180
    if np.nanmax(lon) > 180:
        inv = np.roll(inv, np.round(inv.shape[1]/2).astype(int))
        lon = lon - 180.
    # Done with file, close it
    fid.close()
    
    # Create a Scene if there isn't one
    if ecm_scn is None:
        ecm_scn = Scene()
    
    # Create an area for use in resampling.
    # This assumes lat/lon gridded data
    area_ext = (np.nanmin(lon), np.nanmin(lat), np.nanmax(lon), np.nanmax(lat))
    targ_area = create_area_def("source_area",
                                "EPSG:4326",
                                area_extent=area_ext,
                                width=inv.shape[1],
                                height=inv.shape[0])

    # Create a new dataset in the Scene for the wind data
    ecm_scn[var] = xr.DataArray(da.from_array(inv),
                                coords={'y': lat, 'x': lon},
                                attrs={'start_time': dater})

    ecm_scn[var].attrs['area'] = targ_area
    
    # Return the scene
    return ecm_scn

In [5]:
# Create a scene for the granule and load data
scn = Scene([sev_fname], reader='seviri_l1b_native')
scn.load(bnames, upper_right_corner='NE')

In [6]:
# Load the ECMWF wind
ecm_scn = load_wind(u10_fname, 'u10', ecm_scn=None)
ecm_scn = load_wind(v10_fname, 'v10', ecm_scn=ecm_scn)

In [7]:
# Load the land mask
lsm_scn = Scene([lsm_fname], reader='generic_image')
lsm_scn.load(['image'])

In [8]:
# Resample the ECMWF wind data onto the satellite grid
ecm_scn2 = ecm_scn.resample(scn[bnames[0]].attrs['area'],
                            resampler='nearest', cache_dir=cache_dir,
                            radius_of_influence=150000)

In [9]:
# Copy the winds from the ECMWF scene into the satellite scene.
# This simplifies calling PyCoxMunk, as we only need to pass
# one scene instead of two
scn['u10'] = ecm_scn2['u10'].copy()
scn['v10'] = ecm_scn2['v10'].copy()

In [10]:
# Create the PyCoxMunk class
pcm = pycoxmunk.PyCoxMunk(scn, bnames, angle_names='calc', delete_when_done=False, mask_bad=False, do_brdf=True)

In [11]:
# Set up the wind variables within PyCoxMunk
pcm.setup_wind(scn['u10'], scn['v10'])

In [12]:
# Set up the pixel masking
pcm.setup_pixmask(land_mask=np.array(lsm_scn['image'].data).squeeze())

In [13]:
# Retrieve the sea surface reflectance
pcm.retr_coxmunk_refl()

In [14]:
# Save original data to disk.
pcm.scn.save_datasets(datasets=bnames_out, base_dir=out_dir, enhance=False, dtype=np.float32)